## Semester 1 Project Submission

Please fill out:
* Student name: 
* Instructor name: 


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats

conn = sqlite3.connect('zippedData/im.db')

# Dylan Orndorf-Ronk

# Alex Bonczkiewicz

# Alberto Torres

In [ ]:
q = """
SELECT name 
FROM sqlite_master 
WHERE type='table';
"""
pd.read_sql(q, conn)

In [ ]:
movie_basics = pd.read_sql("""
SELECT * 
FROM movie_basics;
""",conn)
movie_basics

In [ ]:
movie_ratings = pd.read_sql("""
SELECT * 
FROM movie_ratings;
""",conn)
movie_ratings

In [ ]:
movies_df = pd.read_sql("""
SELECT *
FROM movie_akas
INNER JOIN
    movie_basics
ON  movie_akas.movie_id = movie_basics.movie_id
INNER JOIN
    movie_ratings
ON  movie_akas.movie_id = movie_ratings.movie_id
""",conn)
movies_df

In [ ]:
movies_genre = pd.read_sql("""
SELECT genres, sum(numvotes) as num_votes, AVG(averagerating) as avg_rating
FROM movie_basics
INNER JOIN movie_ratings
    ON movie_basics.movie_id = movie_ratings.movie_id
GROUP BY genres
ORDER BY num_votes DESC, avg_rating DESC
LIMIT 20
""",conn)
movies_genre

In [ ]:

movies_df.sort_values(['numvotes','averagerating'], ascending=False).groupby(by="region")

In [ ]:
persons = pd.read_sql("""
SELECT * 
FROM PERSONS;
""",conn)
persons

In [ ]:
directors = pd.read_sql("""
SELECT * 
FROM directors;
""",conn)
directors.head()

In [ ]:
principals = pd.read_sql("""
SELECT * 
FROM principals;
""",conn)
principals

In [ ]:
known_for = pd.read_sql("""
SELECT * 
FROM known_for;
""",conn)
known_for['movie_id'].value_counts()

# Travis McCloughan

# Jaemin Lee